<div style="background-color: #0d0761; border-radius: 20px; color: #fff; font-weight: bold; padding: 10px; text-align:center">
    <h1>Extracción de Datos desde Múltiples Fuentes</h1>
</div>

<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Cargar librerías</h2>
</div>

In [1]:
import pandas as pd
import sqlite3
import json

In [2]:
# crear carpeta para los resultados
import os

ruta_archivos = "../resultados/resultados_dia_2/"
if not os.path.exists(ruta_archivos):
    if not(os.path.exists("../resultados/")): os.mkdir("../resultados/")
    os.mkdir(ruta_archivos)
    print("Carpeta creada")
else:
    print("La carpeta ya existe")

La carpeta ya existe


<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Crear datos de ejemplo en diferentes formatos</h2>
</div>

<div style="font-weight: bold; color:#0d0761 ;  padding: 3px">
    <h3 style="text-decoration:underline; text-decoration-color: #0d0761; text-decoration-thickness:3px">Crear CSV</h3>
</div>

In [3]:
ventas_csv = pd.DataFrame({
    'id_venta': range(1, 6),
    'producto': ['Laptop', 'Mouse', 'Teclado', 'Monitor', 'Audífonos'],
    'precio': [1200, 25, 80, 300, 150]
})
ventas_csv.to_csv(ruta_archivos + 'ventas.csv', index=False)

Pandas usa encoding='uft-8' por defecto, lo que adecuado para el manejo de acentos.

<div style="font-weight: bold; color:#0d0761 ;  padding: 3px">
    <h3 style="text-decoration:underline; text-decoration-color: #0d0761; text-decoration-thickness:3px">Crear Excel con múltiples hojas</h3>
</div>

In [4]:
clientes_df = pd.DataFrame({
    'id_cliente': [1, 2, 3],
    'nombre': ['Ana', 'Carlos', 'María'],
    'ciudad': ['Madrid', 'Barcelona', 'Valencia']
})

with pd.ExcelWriter(ruta_archivos + 'datos.xlsx') as writer:
    ventas_csv.to_excel(writer, sheet_name='Ventas', index=False)
    clientes_df.to_excel(writer, sheet_name='Clientes', index=False)

El uso de context manager garantiza que el archivo se cierre automáticamente al finalizar el bloque with.

openpyxl soporta Unicode de forma nativa por lo que no se requiere especificar encoding para guardar el archivo excel.

<div style="font-weight: bold; color:#0d0761 ;  padding: 3px">
    <h3 style="text-decoration:underline; text-decoration-color: #0d0761; text-decoration-thickness:3px">Crear JSON</h3>
</div>

In [5]:
productos_json = [
    {'id': 101, 'nombre': 'Laptop', 'categoria': 'Electrónica'},
    {'id': 102, 'nombre': 'Mouse', 'categoria': 'Accesorios'}
]
with open(ruta_archivos + 'productos.json', 'w') as f:
    json.dump(productos_json, f)

El uso de context manager garantiza que el archivo se cierre automáticamente al finalizar el bloque with.

No se especificó el encoding, por lo que al guardar el archivo 'productos.json', se usan secuencias de escape para los caracteres con acentos o especiales. Ejemplo: "Electr\u00f3nica" en lugar de "Electrónica".

<img src="../imagenes/productos.PNG" alt="productos.json" width="800"/>

Si se usara:

```python
with open(ruta_archivos + 'productos_2.json', 'w', encoding='utf-8') as f:
    json.dump(productos_json, f, ensure_ascii=False)
```

el archivo 'productos_2.json' registraría la palabra "Electrónica".

In [6]:
with open(ruta_archivos + 'productos_2.json', 'w', encoding='utf-8') as f:
    json.dump(productos_json, f, ensure_ascii=False)

<img src="../imagenes/productos_2.PNG" alt="productos_2.json" width="800"/>

<div style="font-weight: bold; color:#0d0761 ;  padding: 3px">
    <h3 style="text-decoration:underline; text-decoration-color: #0d0761; text-decoration-thickness:3px">Crear base de datos SQLite</h3>
</div>

In [7]:
conn = sqlite3.connect(ruta_archivos + 'ventas.db')
pedidos_df = pd.DataFrame({
    'id_pedido': [1, 2, 3],
    'id_cliente': [1, 2, 1],
    'fecha': ['2024-01-15', '2024-01-16', '2024-01-17'],
    'total': [1225, 25, 380]
})
pedidos_df.to_sql('pedidos', conn, index=False, if_exists='replace')
conn.close()

SQLite usa por defecto Unicode, específicamente UTF-8, aunque también soporta UTF-16BE y UTF-16LE como codificaciones de base de datos internas.

<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Extraer desde cada fuente</h2>
</div>

<div style="font-weight: bold; color:#0d0761 ;  padding: 3px">
    <h3 style="text-decoration:underline; text-decoration-color: #0d0761; text-decoration-thickness:3px">Desde CSV</h3>
</div>

In [8]:
df_csv = pd.read_csv(ruta_archivos + 'ventas.csv')
print("Desde CSV:")
print(df_csv.head())

Desde CSV:
   id_venta   producto  precio
0         1     Laptop    1200
1         2      Mouse      25
2         3    Teclado      80
3         4    Monitor     300
4         5  Audífonos     150


El dataframe obtenido es consistente.

<div style="font-weight: bold; color:#0d0761 ;  padding: 3px">
    <h3 style="text-decoration:underline; text-decoration-color: #0d0761; text-decoration-thickness:3px">Desde Excel (hoja específica)</h3>
</div>

In [9]:
df_excel_ventas = pd.read_excel(ruta_archivos + 'datos.xlsx', sheet_name='Ventas')
df_excel_clientes = pd.read_excel(ruta_archivos + 'datos.xlsx', sheet_name='Clientes')
print("\nDesde Excel - Ventas:")
print(df_excel_ventas.head())


Desde Excel - Ventas:
   id_venta   producto  precio
0         1     Laptop    1200
1         2      Mouse      25
2         3    Teclado      80
3         4    Monitor     300
4         5  Audífonos     150


In [10]:
print(df_excel_clientes.head())

   id_cliente  nombre     ciudad
0           1     Ana     Madrid
1           2  Carlos  Barcelona
2           3   María   Valencia


Los dataframes obtenidos son consistentes.

<div style="font-weight: bold; color:#0d0761 ;  padding: 3px">
    <h3 style="text-decoration:underline; text-decoration-color: #0d0761; text-decoration-thickness:3px">Desde JSON</h3>
</div>

In [11]:
df_json = pd.read_json(ruta_archivos + 'productos.json')
print("\nDesde JSON:")
print(df_json)


Desde JSON:
    id  nombre    categoria
0  101  Laptop  Electrónica
1  102   Mouse   Accesorios


Pandas usa encoding='uft-8' por defecto, lo que adecuado para el manejo de acentos.

El dataframe obtenido es consistente.

<div style="font-weight: bold; color:#0d0761 ;  padding: 3px">
    <h3 style="text-decoration:underline; text-decoration-color: #0d0761; text-decoration-thickness:3px">Desde SQLite</h3>
</div>

In [12]:
# sin context manager
conn = sqlite3.connect(ruta_archivos + 'ventas.db')
df_sql = pd.read_sql('SELECT * FROM pedidos', conn)
conn.close()
print("\nDesde SQLite:")
print(df_sql)


Desde SQLite:
   id_pedido  id_cliente       fecha  total
0          1           1  2024-01-15   1225
1          2           2  2024-01-16     25
2          3           1  2024-01-17    380


In [13]:
# con context manager
with sqlite3.connect(ruta_archivos + 'ventas.db') as conn2:
    df_sql2 = pd.read_sql('SELECT * FROM pedidos', conn2)
print("\nDesde SQLite con context manager:")
print(df_sql2)


Desde SQLite con context manager:
   id_pedido  id_cliente       fecha  total
0          1           1  2024-01-15   1225
1          2           2  2024-01-16     25
2          3           1  2024-01-17    380


En ambos casos, el dataframe obtenido es consistente.

<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>API simulada</h2>
</div>

In [14]:
# Simular API response
api_response = {
    'status': 'success',
    'data': [
        {'id': 201, 'producto': 'Webcam', 'stock': 15},
        {'id': 202, 'producto': 'Micrófono', 'stock': 8}
    ]
}

# Simular consumo de API
df_api = pd.DataFrame(api_response['data'])
print("\nDesde API simulada:")
print(df_api)


Desde API simulada:
    id   producto  stock
0  201     Webcam     15
1  202  Micrófono      8


El dataframe obtenido es consistente.